In [5]:
import os
import numpy as np
import librosa
import joblib
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import plot_model


In [7]:
dataset_path = "audio_files"

emotion_map = {
    '01': 'neutral',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry'
}

features = []
labels = []


In [1]:
def extract_features(data, sr):
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result = np.hstack((result, zcr))
    stft = np.abs(librosa.stft(data))
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
    result = np.hstack((result, chroma))
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sr).T, axis=0)
    result = np.hstack((result, mfcc))
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms))
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sr).T, axis=0)
    result = np.hstack((result, mel))
    return result


In [9]:
for actor in os.listdir(dataset_path):
    actor_folder = os.path.join(dataset_path, actor)
    if not os.path.isdir(actor_folder):
        continue

    for file in os.listdir(actor_folder):
        if not file.endswith(".wav"):
            continue

        emotion_code = file.split("-")[2]
        emotion = emotion_map.get(emotion_code)
        if emotion is None:
            continue

        file_path = os.path.join(actor_folder, file)
        try:
            data, sr = librosa.load(file_path, duration=2.5, offset=0.6)
            feat = extract_features(data, sr)
            features.append(feat)
            labels.append(emotion)
            print(f"✅ Processed: {file_path}")
        except Exception as e:
            print(f"❌ Error: {file_path} - {e}")


C:\Users\Dell\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


✅ Processed: audio_files\Actor_01\03-01-01-01-01-01-01.wav
✅ Processed: audio_files\Actor_01\03-01-01-01-01-02-01.wav
✅ Processed: audio_files\Actor_01\03-01-01-01-02-01-01.wav
✅ Processed: audio_files\Actor_01\03-01-01-01-02-02-01.wav
✅ Processed: audio_files\Actor_01\03-01-03-01-01-01-01.wav
✅ Processed: audio_files\Actor_01\03-01-03-01-01-02-01.wav
✅ Processed: audio_files\Actor_01\03-01-03-01-02-01-01.wav
✅ Processed: audio_files\Actor_01\03-01-03-01-02-02-01.wav
✅ Processed: audio_files\Actor_01\03-01-03-02-01-01-01.wav
✅ Processed: audio_files\Actor_01\03-01-03-02-01-02-01.wav
✅ Processed: audio_files\Actor_01\03-01-03-02-02-01-01.wav
✅ Processed: audio_files\Actor_01\03-01-03-02-02-02-01.wav
✅ Processed: audio_files\Actor_01\03-01-04-01-01-01-01.wav
✅ Processed: audio_files\Actor_01\03-01-04-01-01-02-01.wav
✅ Processed: audio_files\Actor_01\03-01-04-01-02-01-01.wav
✅ Processed: audio_files\Actor_01\03-01-04-01-02-02-01.wav
✅ Processed: audio_files\Actor_01\03-01-04-02-01-01-01.w

In [11]:
X = np.array(features)
y = np.array(labels)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))


In [13]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_scaled.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_encoded.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_scaled, y_encoded, epochs=50, batch_size=32, validation_split=0.2)


C:\Users\Dell\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.3472 - loss: 1.4965 - val_accuracy: 0.5111 - val_loss: 1.0762
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4170 - loss: 1.3159 - val_accuracy: 0.5185 - val_loss: 1.0500
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4901 - loss: 1.0783 - val_accuracy: 0.5333 - val_loss: 1.0121
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5054 - loss: 1.0205 - val_accuracy: 0.5185 - val_loss: 0.9840
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5761 - loss: 0.9801 - val_accuracy: 0.5481 - val_loss: 0.9864
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6612 - loss: 0.8449 - val_accuracy: 0.5111 - val_loss: 0.9582
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5926 - loss: 0.8994 - val_accuracy: 0.5333 - val_loss: 0.9748
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6597 - loss: 0.8189 - val_accuracy: 0.5259 - va

In [15]:
model.save("emotion_model.h5")
joblib.dump(scaler, "standard_scaler.save")
joblib.dump(encoder, "onehot_encoder.save")


['onehot_encoder.save']

In [19]:
!pip install streamlit-mic-recorder


   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
    --------------------------------------- 0.0/2.2 MB ? eta -:--:--
    --------------------------------------- 0.0/2.2 MB ? eta -:--:--
    --------------------------------------- 0.0/2.2 MB 279.3 kB/s eta 0:00:08
    --------------------------------------- 0.0/2.2 MB 279.3 kB/s eta 0:00:08
    --------------------------------------- 0.0/2.2 MB 279.3 kB/s eta 0:00:08
   - -------------------------------------- 0.1/2.2 MB 348.6 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.2 MB 348.6 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.2 MB 348.6 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.2 MB 348.6 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.2 MB 348.6 kB/s eta 0:00:07
   --- ------------------------------------ 0.2/2.2 MB 389.1 kB/s eta 0:00:06
   --- ------------------------------------ 0.2/2.2 MB 389.1 kB/s eta 0:00:06
   --- ------

In [21]:
!pip install soundfile

In [23]:
!pip install pydub

In [3]:
import os
print(os.path.exists(r"C:\ffmpeg\bin\ffmpeg.exe"))  # Should return True


True


In [1]:

import shutil
print(shutil.which("ffmpeg"))


C:\ffmpeg\bin\ffmpeg.EXE
